In [1]:
import numpy
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType
import onnx

In [2]:
corpus = numpy.array([
            "This is the first document.",
            "This document is the second document.",
            "And this is the third one.",
            "Is this the first document?",
        ]).reshape((4, 1))

In [3]:
vect = HashingVectorizer(ngram_range=(1, 1))
cvect = CountVectorizer(ngram_range=(1, 1))

In [4]:
Y = cvect.fit_transform(corpus.ravel())
print(cvect.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [5]:
X = vect.fit_transform(corpus.ravel())
print(X.shape)

(4, 1048576)


In [6]:
model_onnx_c = convert_sklearn(cvect, "CountVectorizer",
                                     [("input", StringTensorType([1]))])

/home/realno/sklearn-onnx/skl2onnx/operator_converters/text_vectoriser.py:171: UserWarning: Converter for TfidfVectorizer will use scikit-learn regular expression by default in version 1.6.
  UserWarning)
/home/realno/sklearn-onnx/skl2onnx/common/_container.py:519: UserWarning: Unable to find operator 'Tokenizer' in domain 'com.microsoft' in ONNX, op_version is forced to 1.
  op_type, domain))


In [7]:
model_onnx = convert_sklearn(vect, "HashingVectorizer",
                                     [("input", StringTensorType([1]))])

/home/realno/sklearn-onnx/skl2onnx/operator_converters/hashing_vectoriser.py:102: UserWarning: Converter for HashingVectorizer will use scikit-learn regular expression by default in version 1.6.
  UserWarning)


In [8]:
print(model_onnx)

ir_version: 7
producer_name: "skl2onnx"
producer_version: "1.6.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "input"
    input: "shape_tensor"
    output: "flattened"
    name: "Reshape"
    op_type: "Reshape"
    domain: ""
  }
  node {
    input: "flattened"
    output: "normalized"
    name: "StringNormalizer"
    op_type: "StringNormalizer"
    attribute {
      name: "case_change_action"
      s: "LOWER"
      type: STRING
    }
    attribute {
      name: "is_case_sensitive"
      i: 0
      type: INT
    }
    domain: ""
  }
  node {
    input: "normalized"
    output: "tokenized"
    name: "Tokenizer"
    op_type: "Tokenizer"
    attribute {
      name: "mark"
      i: 0
      type: INT
    }
    attribute {
      name: "mincharnum"
      i: 1
      type: INT
    }
    attribute {
      name: "pad_value"
      s: "#"
      type: STRING
    }
    attribute {
      name: "tokenexp"
      s: "[a-zA-Z0-9_]+"
      type: STRING
    }
    domain: "c